# The Logic 

1. We will be using the sloth to finetune the model 
2. PEFT will allow us to add LoRA adapters which will allow us to finetune the model on a smaller dataset
3. TRT will allow us to add the techinical configurations needed
4. We will be using the HuggingFace Hub to store the model

### Load training data

In [3]:
import json

file = json.load(open("training.json", "r"))
print(file[1])

{'category': 'immigration', 'text': 'Border security is national security - we need complete operational control of our southern border.'}


### Install all dependencies

In [1]:
%pip uninstall -y unsloth peft
%pip install unsloth trl peft accelerate bitsandbytes

Note: you may need to restart the kernel to use updated packages.
  Using cached unsloth-2025.11.3-py3-none-any.whl.metadata (61 kB)
  Using cached trl-0.25.1-py3-none-any.whl.metadata (11 kB)
  Using cached peft-0.18.0-py3-none-any.whl.metadata (14 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached bitsandbytes-0.42.0-py3-none-any.whl.metadata (9.9 kB)
  Using cached unsloth_zoo-2025.11.4-py3-none-any.whl.metadata (32 kB)
  Using cached tyro-0.9.35-py3-none-any.whl.metadata (12 kB)
INFO: pip is looking at multiple versions of unsloth to determine which version is compatible with other requirements. This could take a while.
  Using cached unsloth-2025.11.2-py3-none-any.whl.metadata (61 kB)
  Using cached unsloth-2025.11.1-py3-none-any.whl.metadata (61 kB)
  Using cached unsloth-2025.10.12-py3-none-any.whl.metadata (61 kB)
  Using cached unsloth-2025.10.11-py3-none-any.whl.metadata (61 kB)
  Using cached unsloth-2025.10.10-py3-none-any.whl.metadata (61

### Load pretrained model (without fine tuning)

In [1]:
from unsloth import FastLanguageModel

model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

ModuleNotFoundError: No module named 'unsloth'

### Prepare dataset for finetuning
Ensure that the LLM knows what is expected input and output by specifying the format of the input and output.

In [2]:
from datasets import Dataset

def format_prompt(example):
    return f"### Input: {example['text']}\n### Output: {example['category']}<|endoftext|>"

formatted_data = [format_prompt(item) for item in file]
dataset = Dataset.from_dict({"text": formatted_data})

ModuleNotFoundError: No module named 'datasets'

### Add LoRA adapters 
LoRA adapters is a techinique to add a small amount of parameters to the model to improve its performance on a specific task.

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

### Add technical configurations to the model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
    ),
)

### Train the model

In [ ]:
trainer_stats = trainer.train()